In [1]:
import os
os.listdir('./')

['test_set_router_result.csv',
 'run_validation.ipynb',
 'test_set_router.csv',
 'test_classification_set.numbers']

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import asyncio
import aiohttp

import sys
sys.path.append('../../')

from rag import route_and_rag

test_df = pd.read_csv('./test_set_router.csv')
test_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,question,label
0,Кто такой Руслан масгутов?,Информация о курсе/Наша команда.txt
1,Какие кураторы на курсе?,Информация о курсе/Наша команда.txt
2,Есть ли врачи на курсе?,Информация о курсе/Наша команда.txt
3,Можно ли оливки на второй фазе?,Питание/Принципы питания.txt
4,Как правильно питаться?,Питание/Принципы питания.txt
5,Можно ли есть бобовые?,Питание/Принципы питания.txt
6,Что ты умеешь?,Информация о курсе/Информация о тебе.txt
7,Кто тебя разработал?,Информация о курсе/Информация о тебе.txt
8,Зачем мне проходить курс?,Информация о курсе/Общая информация.txt
9,Какие практики есть на курсе?,Информация о курсе/Общая информация.txt


In [2]:
# for idx, row in test_df.iterrows():
#     response = await route_and_rag(row['question'], router_model="gpt-4o-mini")
#     test_df.iloc[idx, test_df.columns.get_loc('choosed_chapters')] = response['choosed_chapters']
#     test_df.iloc[idx, test_df.columns.get_loc('response_text')] = response['response_text']



In [3]:
class AttrDict(dict):
    def __getattr__(self, item):
        try:
            return self[item]
        except KeyError:
            raise AttributeError(f"'AttrDict' object has no attribute '{item}'")
    
    def __setattr__(self, key, value):
        self[key] = value
    
    def __delattr__(self, item):
        try:
            del self[item]
        except KeyError:
            raise AttributeError(f"'AttrDict' object has no attribute '{item}'")

# Usage
context = AttrDict({
    'user_data': AttrDict({'input_message': 'asd'})
})

context.user_data['input_message']

'asd'

In [4]:
async def process_row(test_df, idx, row):
    context = AttrDict({
        'user_data': AttrDict({'input_message': row['question']})
    })

    response = await route_and_rag(context, [], router_model="gpt-4o-mini")
    if 'choosed_chapters' in response.keys():
        test_df.iloc[idx, test_df.columns.get_loc('choosed_chapters')] = response['choosed_chapters']
        test_df.iloc[idx, test_df.columns.get_loc('response_text')] = response['response_text']

# URL for the route_and_rag endpoint
url = 'http://example.com/api/route_and_rag'

# Add columns to the DataFrame
test_df['choosed_chapters'] = ""
test_df['response_text'] = ""


async with aiohttp.ClientSession() as session:
    tasks = []
    for idx, row in test_df.iterrows():
        task = asyncio.create_task(process_row(test_df, idx, row))
        tasks.append(task)
    await asyncio.gather(*tasks)

In [6]:
test_df

,question,label,choosed_chapters,response_text
0,Кто такой Руслан масгутов?,Информация о курсе/Наша команда.txt,[Информация о курсе/Наша команда.txt],"1. Руслан Фаридович Масгутов является ""Руковод..."
1,Какие кураторы на курсе?,Информация о курсе/Наша команда.txt,[Информация о курсе/Наша команда.txt],1. Куратор - Кермакунова Айгерим Карагуловна -...
2,Есть ли врачи на курсе?,Информация о курсе/Наша команда.txt,[Информация о курсе/Наша команда.txt],"1. Да, на курсе есть врачи. Например, ""Руковод..."
3,Можно ли оливки на второй фазе?,Питание/Принципы питания.txt,[Питание/Принципы питания.txt],"1. В контексте упоминается, что ""ферментирован..."
4,Как правильно питаться?,Питание/Принципы питания.txt,[Питание/Принципы питания.txt],"1. ""На курсе основополагающим правилом питания..."
5,Можно ли есть бобовые?,Питание/Принципы питания.txt,[Питание/Принципы питания.txt],"1. ""Все виды бобовых, за счет высокого содержа..."
6,Что ты умеешь?,Информация о курсе/Информация о тебе.txt,[Информация о курсе/Информация о тебе.txt],"1. ""Ты персональный ассистент на курсе 'Здоров..."
7,Кто тебя разработал?,Информация о курсе/Информация о тебе.txt,[Информация о курсе/Информация о тебе.txt],"1. ""Ты была разработана очень талантливой кома..."
8,Зачем мне проходить курс?,Информация о курсе/Общая информация.txt,[Информация о курсе/Общая информация.txt],"1. Курс ""Здоровье без лекарств: научный подход..."
9,Какие практики есть на курсе?,Информация о курсе/Общая информация.txt,[Информация о курсе/Общая информация.txt],"1. ""Курс предлагает не только теоретические зн..."


In [12]:
test_df.to_csv('test_set_router_result.csv')
test_df

,question,label,choosed_chapters,response_text
0,Кто такой Руслан масгутов?,Информация о курсе/Наша команда.txt,[Информация о курсе/Наша команда.txt],"1. Руслан Фаридович Масгутов является ""Руковод..."
1,Какие кураторы на курсе?,Информация о курсе/Наша команда.txt,[Информация о курсе/Наша команда.txt],1. Куратор - Кермакунова Айгерим Карагуловна -...
2,Есть ли врачи на курсе?,Информация о курсе/Наша команда.txt,[Информация о курсе/Наша команда.txt],"1. Да, на курсе есть врачи. Например, ""Руковод..."
3,Можно ли оливки на второй фазе?,Питание/Принципы питания.txt,[Питание/Принципы питания.txt],"1. В контексте упоминается, что ""продукты, сни..."
4,Как правильно питаться?,Питание/Принципы питания.txt,[Питание/Принципы питания.txt],"1. ""На курсе основополагающим правилом питания..."
5,Можно ли есть бобовые?,Питание/Принципы питания.txt,[Питание/Принципы питания.txt],"1. ""Все виды бобовых, за счет высокого содержа..."
6,Что ты умеешь?,Информация о курсе/Информация о тебе.txt,[Информация о курсе/Информация о тебе.txt],"1. ""Ты персональный ассистент на курсе 'Здоров..."
7,Кто тебя разработал?,Информация о курсе/Информация о тебе.txt,[Информация о курсе/Наша команда.txt],The context does not provide specific informat...
8,Зачем мне проходить курс?,Информация о курсе/Общая информация.txt,[Информация о курсе/Общая информация.txt],"1. Курс ""Здоровье без лекарств: научный подход..."
9,Какие практики есть на курсе?,Информация о курсе/Общая информация.txt,[Информация о курсе/Общая информация.txt],"1. ""Курс предлагает не только теоретические зн..."


In [23]:
test_df[['question', 'label', 'choosed_chapters']].to_dict(orient='records')

[{'question': 'Кто такой Руслан масгутов?',
  'label': 'Информация о курсе/Наша команда.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Какие кураторы на курсе?',
  'label': 'Информация о курсе/Наша команда.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Есть ли врачи на курсе?',
  'label': 'Информация о курсе/Наша команда.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Можно ли оливки на второй фазе?',
  'label': 'Питание/Принципы питания.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Как правильно питаться?',
  'label': 'Питание/Принципы питания.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Можно ли есть бобовые?',
  'label': 'Питание/Принципы питания.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Что ты умеешь?',
  'label': 'Информация о курсе/Информация о тебе.txt',
  'choosed_cha

In [ ]:
ToC is a one-shot LLM-classifier instructions for router model which selected appropriate documents for user question.
Test dataset contains user qestion, true label (document that need to be chosen) and choosed_chapters are results of classifier (it should ideally be same as label). Adapt descriptions of classes in ToC so that LLM can handle classification more accurately. Try to describe it in more general words so that model can approximate future unknown requests

ToC = {
    "Питание/Принципы питания.txt": "совместимость продуктов, правило тарелки, программа питания на курсе, что можно кушать на второй фазе, список разрешенных продуктов, description of image with food",
    "Информация о курсе/Наша команда.txt": "перечислены и описаны все авторы и кураторы курса, используй для ответа на вопросы о конкретных личностях",
    "Информация о курсе/Общая информация.txt": "что есть на курсе, кому он нужен, какие проблемы решает",
    "Информация о курсе/Информация о тебе.txt": "кто ты такая, что ты умеешь, описание ИИ ассистента",
    "Информация о курсе/Ссылки по курсу.txt": "структура курса, про уроки, ссылки на уроки, где какие задания и гайды находятся, используй для ответа на вопросы о том где найти те или иные уроки и задания"
}



test_dataset = [{'question': 'Кто такой Руслан масгутов?',
  'label': 'Информация о курсе/Наша команда.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Какие кураторы на курсе?',
  'label': 'Информация о курсе/Наша команда.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Есть ли врачи на курсе?',
  'label': 'Информация о курсе/Наша команда.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Можно ли оливки на второй фазе?',
  'label': 'Питание/Принципы питания.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Как правильно питаться?',
  'label': 'Питание/Принципы питания.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Можно ли есть бобовые?',
  'label': 'Питание/Принципы питания.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Что ты умеешь?',
  'label': 'Информация о курсе/Информация о тебе.txt',
  'choosed_chapters': ['Информация о курсе/Информация о тебе.txt']},
 {'question': 'Кто тебя разработал?',
  'label': 'Информация о курсе/Информация о тебе.txt',
  'choosed_chapters': ['Информация о курсе/Наша команда.txt']},
 {'question': 'Зачем мне проходить курс?',
  'label': 'Информация о курсе/Общая информация.txt',
  'choosed_chapters': ['Информация о курсе/Общая информация.txt']},
 {'question': 'Какие практики есть на курсе?',
  'label': 'Информация о курсе/Общая информация.txt',
  'choosed_chapters': ['Информация о курсе/Общая информация.txt']},
 {'question': 'Что такое здоровье без лекарств?',
  'label': 'Информация о курсе/Общая информация.txt',
  'choosed_chapters': ''},
 {'question': 'Где найти анкету Точка А?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ['Информация о курсе/Ссылки по курсу.txt']},
 {'question': 'Где найти промокод чтобы подключить Аику?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Почему в модуле 0 у меня только один урок?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ['Информация о курсе/Ссылки по курсу.txt']},
 {'question': 'Где найд по мозговой активности?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Зачем мне пить препараты из гайда по мозговой активности?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Где список анализов?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Где купон со скидкой?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Куда прикрепить результаты анализов?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'В каком уроке техника/практика?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ['Информация о курсе/Ссылки по курсу.txt']},
 {'question': 'Как питаться после сдачи анализов?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ['Питание/Принципы питания.txt']},
 {'question': 'Где найти таблицу противовоспалительных препаратов?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Когда открывается Модуль 5?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ['Информация о курсе/Ссылки по курсу.txt']},
 {'question': 'Когда открывается магазин подарков?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Почему урок не открывается?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ['Информация о курсе/Ссылки по курсу.txt']},
 {'question': 'Препараты еще не куплены как заполнять ежедневные отчеты?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''},
 {'question': 'Сколько у меня времени досдать фото После?',
  'label': 'Информация о курсе/Ссылки по курсу.txt',
  'choosed_chapters': ''}]

In [ ]:
# OLD
// {
//     "Питание/Принципы питания.txt": "Руководства по питанию, включая совместимость продуктов, планирование питания, правило тарелки и список разрешенных продуктов на различных этапах курса. Также включает описание изображений с едой.",
//     "Информация о курсе/Наша команда.txt": "“Информация о команде курса, включая описания всех авторов, кураторов и других связанных профессионалов. Используй для поиска информации на вопросы о конкретных личностях",
//     "Информация о курсе/Общая информация.txt": "“Общий обзор курса, включая его цель, целевую аудиторию и решаемые проблемы. Используй для вопросов, касающихся целей и преимуществ курса.",
//     "Информация о курсе/Информация о тебе.txt": "“Информация об ИИ-ассистенте, включая его возможности, функции и роль в помощи пользователю. Используй для вопросов о личности ассистента или его функционале.",
//     "Информация о курсе/Ссылки по курсу.txt": "“Ссылки и ресурсы, связанные со структурой курса, уроками, заданиями, гайдами, промокодами. Используй для вопросов типа 'где найти что-то? (ссы' "
// }

